In [1]:
pip install labelImg

  Using cached labelImg-1.8.6-py2.py3-none-any.whl
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------- -------------------------------- 1.3/6.9 MB 8.4 MB/s eta 0:00:01
   ------------------------ --------------- 4.2/6.9 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 12.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   --- ------------------------------------ 3.9/50.1 MB 19.6 MB/s eta 0:00:03
   ------ --------------------------------- 7.6/50.1 MB 19.6 MB/s eta 0:00:03
   ------- -------------------------------- 9.7/50.1 MB 15.9 MB/s eta 0:00:03
   --------- ------------------------------ 11.8/50.1 MB 14.5 MB/s eta 0:00:03
   ----------- ---------------------------- 14.4/50.1 MB 13.9 MB/s eta 0:00:03
   -------------- ------------------------- 18.6/50.1 MB 15.3 MB/s eta 0:00:03
   ----------------- ---------------------- 21.5/50.1 MB 15.3 MB/s eta 0:00:02
   -------------------- ------

In [2]:
# Import subprocess to run external applications
import subprocess
import os

# Step 1: Define the path to your image directory
image_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\images'  # Replace this with your actual image folder path

# Step 2: Define the path to your label (YOLO txt files) directory
label_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\labels'  # Replace this with your actual labels folder path

# Step 3: Check if labelImg is installed in the environment
try:
    # Try to run labelImg directly
    subprocess.run(['labelImg', image_dir, label_dir])
except FileNotFoundError:
    print("labelImg is not found in your environment.")
    print("Install it by running: pip install labelImg")


In [4]:
import os
import subprocess

# ----------------- SET YOUR PATHS HERE -----------------
image_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\images'     # Image folder
label_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\labels'     # YOLO label folder
num_classes = 1  # Adjust based on your dataset (e.g., weapon = 1 class)
# -------------------------------------------------------

def is_label_valid(label_line, line_num, file_path):
    """Check if a single YOLO label line is valid (correct format and value range)."""
    parts = label_line.strip().split()

    # Each YOLO line must have 5 values: class x y width height
    if len(parts) != 5:
        return f"[{file_path}] Line {line_num}: Expected 5 values, got {len(parts)}"

    try:
        cls, x, y, w, h = map(float, parts)
        if not (0 <= cls < num_classes):
            return f"[{file_path}] Line {line_num}: Invalid class index {cls}"
        if not all(0 <= v <= 1 for v in [x, y, w, h]):
            return f"[{file_path}] Line {line_num}: One or more coordinates out of range (0–1)"
    except ValueError:
        return f"[{file_path}] Line {line_num}: Non-numeric values found"

    return None

def check_labels(label_dir):
    """Scan all YOLO labels in the directory and return list of errors."""
    errors = []
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            path = os.path.join(label_dir, filename)
            with open(path, 'r') as f:
                for i, line in enumerate(f.readlines(), 1):
                    err = is_label_valid(line, i, filename)
                    if err:
                        errors.append(err)
    return errors

# Step 1: Check for label issues
print("🔍 Checking labels...")
issues = check_labels(label_dir)

# Step 2: Print summary and details
if issues:
    print(f"\n❌ {len(issues)} label issue(s) found:")
    for issue in issues:
        print(issue)
else:
    print("✅ All labels appear valid.")

# Step 3: Launch labelImg for manual re-check
try:
    print("\n🚀 Launching labelImg...")
    subprocess.run(['labelImg', image_dir, label_dir])
except FileNotFoundError:
    print("\n⚠️ 'labelImg' is not installed.")
    print("➡️  Install it using: pip install labelImg")


🔍 Checking labels...

❌ 3 label issue(s) found:
[00000661_jpg.rf.6412ab15b6b014ba8da497ae14b20594.txt] Line 1: Expected 5 values, got 11
[00000661_jpg.rf.b14a643431bc7b6051f5dbabe6bac63e.txt] Line 1: Expected 5 values, got 11
[00000661_jpg.rf.e980191d99adc63c68acb40e55793cd7.txt] Line 1: Expected 5 values, got 11

🚀 Launching labelImg...


In [5]:
import os
import subprocess

# ----------------- SET YOUR PATHS HERE -----------------
image_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\images'     # Image folder
label_dir = r'C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\labels'     # Label folder
num_classes = 1  # Set number of valid classes (e.g., weapon = 1 class)
# -------------------------------------------------------

def is_label_valid(label_line, line_num, file_path):
    """Check if a single YOLO label line is valid."""
    parts = label_line.strip().split()

    if len(parts) != 5:
        return f"[{file_path}] Line {line_num}: Expected 5 values, got {len(parts)}"

    try:
        cls, x, y, w, h = map(float, parts)
        if not (0 <= cls < num_classes):
            return f"[{file_path}] Line {line_num}: Invalid class index {cls}"
        if not all(0 <= v <= 1 for v in [x, y, w, h]):
            return f"[{file_path}] Line {line_num}: One or more coordinates out of range (0–1)"
    except ValueError:
        return f"[{file_path}] Line {line_num}: Non-numeric values found"

    return None

def check_labels_and_collect_errors(label_dir):
    """Check label files and return (errors, files with errors)."""
    errors = []
    files_with_issues = set()
    
    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            path = os.path.join(label_dir, filename)
            with open(path, 'r') as f:
                for i, line in enumerate(f.readlines(), 1):
                    err = is_label_valid(line, i, filename)
                    if err:
                        errors.append(err)
                        files_with_issues.add(os.path.splitext(filename)[0])  # Add file name without extension

    return errors, files_with_issues

def remove_invalid_files(file_basenames, image_dir, label_dir):
    """Delete both image and label file for invalid annotations."""
    for base in file_basenames:
        image_path_jpg = os.path.join(image_dir, base + ".jpg")
        image_path_png = os.path.join(image_dir, base + ".png")
        label_path = os.path.join(label_dir, base + ".txt")
        
        for img_path in [image_path_jpg, image_path_png]:
            if os.path.exists(img_path):
                os.remove(img_path)
                print(f"🗑️ Removed image: {img_path}")
        
        if os.path.exists(label_path):
            os.remove(label_path)
            print(f"🗑️ Removed label: {label_path}")

# Step 1: Check labels
print("🔍 Checking labels...")
issues, files_with_issues = check_labels_and_collect_errors(label_dir)

# Step 2: Report issues
if issues:
    print(f"\n❌ {len(issues)} label issue(s) found:")
    for issue in issues:
        print(issue)

    print(f"\n🧹 Removing {len(files_with_issues)} corrupted image-label pair(s)...")
    remove_invalid_files(files_with_issues, image_dir, label_dir)
else:
    print("✅ All labels appear valid.")

# Step 3: Launch labelImg for manual check
try:
    print("\n🚀 Launching labelImg...")
    subprocess.run(['labelImg', image_dir, label_dir])
except FileNotFoundError:
    print("\n⚠️ 'labelImg' is not installed.")
    print("➡️  Install it using: pip install labelImg")


🔍 Checking labels...

❌ 3 label issue(s) found:
[00000661_jpg.rf.6412ab15b6b014ba8da497ae14b20594.txt] Line 1: Expected 5 values, got 11
[00000661_jpg.rf.b14a643431bc7b6051f5dbabe6bac63e.txt] Line 1: Expected 5 values, got 11
[00000661_jpg.rf.e980191d99adc63c68acb40e55793cd7.txt] Line 1: Expected 5 values, got 11

🧹 Removing 3 corrupted image-label pair(s)...
🗑️ Removed image: C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\images\00000661_jpg.rf.e980191d99adc63c68acb40e55793cd7.jpg
🗑️ Removed label: C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\labels\00000661_jpg.rf.e980191d99adc63c68acb40e55793cd7.txt
🗑️ Removed image: C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\images\00000661_jpg.rf.b14a643431bc7b6051f5dbabe6bac63e.jpg
🗑️ Removed label: C:\Users\dsy92\OneDrive\Desktop\Weapon-Detection\enhance_image\train\labels\00000661_jpg.rf.b14a643431bc7b6051f5dbabe6bac63e.txt
🗑️ Removed image: C:\Users\dsy92\OneDrive\Desktop\